In [ ]:
%pip install -qU --upgrade transformers evaluate peft seqeval sentencepiece accelerate datasets bitsandbytes rjieba huggingface_hub
import pandas as pd
import regex as re
import numpy as np
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from datasets import load_dataset
# hiner = load_dataset('cfilt/HiNER-collapsed', streaming=True)
hiner = load_dataset('cfilt/HiNER-original')

label_list = ["B-LOCATION", "B-ORGANIZATION", "B-PERSON", "I-LOCATION", "I-ORGANIZATION","I-PERSON","O"]

label_list = ["B-FESTIVAL","B-GAME","B-LANGUAGE","B-LITERATURE","B-LOCATION","B-MISC","B-NUMEX","B-ORGANIZATION",
              "B-PERSON","B-RELIGION","B-TIMEX","I-FESTIVAL","I-GAME","I-LANGUAGE","I-LITERATURE","I-LOCATION",
              "I-MISC","I-NUMEX","I-ORGANIZATION","I-PERSON","I-RELIGION","I-TIMEX","O"]

id2label = {idx:i for idx, i in enumerate(label_list)}
label2id = {i: idx for idx, i in enumerate(label_list)}

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
vocab = set()
LEN = 0
for i in tqdm(iter(hiner['train'])):
    vocab = vocab | set(i["tokens"])
    LEN += 1
# len(vocab)

75827it [02:25, 522.00it/s]


In [ ]:
from transformers import AutoTokenizer
TOKENIZERNAME = "monsoon-nlp/hindi-bert"
MODELNAME = "xlm-roberta-large"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZERNAME)
# tokenizer = AutoTokenizer.from_pretrained(MODELNAME)

# tokenizer.model_max_length = tokenizer2.model_max_length
tokenizer

# tokenizer = tokenizer.train_new_from_iterator(hiner["train"]["tokens"], vocab_size = 100000)
# tokenizer.pad_token = tokenizer.eos_token
new_tokens = vocab - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens), special_tokens=True)

58934

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/NLP/DistilBERT/Tokenizer/", use_fast = False)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,padding=True,is_split_into_words=True)
    # tokenized_inputs = tokenizer(examples["tokens"], padding="max_length", truncation=True, max_length=230, return_tensors="pt", is_split_into_words=True, return_token_type_ids=False)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = hiner["train"].map(tokenize_and_align_labels, batched=True)
tokenized_validation = hiner['validation'].map(tokenize_and_align_labels, batched=True)
tokenized_test = hiner['test'].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/75827 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/10851 [00:00<?, ? examples/s]

Map:   0%|          | 0/21657 [00:00<?, ? examples/s]

In [ ]:
y = 78
# list(zip(tokenized_validation["tokens"][y],tokenized_validation["ner_tags"][y]))

for a,b,c,d in (tokenized_validation["input_ids"][y].word_ids,tokenizer.convert_ids_to_tokens(tokenized_validation["input_ids"][y]),tokenized_validation["input_ids"][y],tokenized_validation["labels"][y]):
  print(a,b,c,d)

AttributeError: ignored

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import numpy as np

import evaluate
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels =p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AlbertForTokenClassification, BloomForTokenClassification, XLNetForTokenClassification, ConvBertForTokenClassification, BitsAndBytesConfig
import torch


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# model = AutoModelForTokenClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id
# )


model = AutoModelForTokenClassification.from_pretrained(
    MODELNAME, num_labels=len(label_list), id2label=id2label, label2id=label2id, ignore_mismatched_sizes = True# load_in_8bit = True # quantization_config = bnb_config
)

# model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP/DistilBERT/Model")


# model = AlbertForTokenClassification.from_pretrained(
#     "albert-base-v2", num_labels=2, id2label=id2label, label2id=label2id
# )

# model = BloomForTokenClassification.from_pretrained("bigscience/bloom-560m", num_labels=2, id2label=id2label, label2id=label2id)

# model = XLNetForTokenClassification.from_pretrained("xlnet-base-cased", num_labels=2, id2label=id2label, label2id=label2id)

# model = ConvBertForTokenClassification.from_pretrained("YituTech/conv-bert-base", num_labels=3, id2label=id2label, label2id=label2id)

model.resize_token_embeddings(len(tokenizer))

Embedding(324721, 1024, padding_idx=1)

In [ ]:
# @title PEFT
# from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
# model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(task_type=TaskType.TOKEN_CLS,
target_modules=["q_lin", "k_lin", "v_lin", "out_lin"], inference_mode=False, r=8, lora_alpha=16,
lora_dropout=0.1, bias="lora_only", modules_to_save = ['word_embeddings', "classifier"])


# model = get_peft_model(model, peft_config)

# model.model.distilbert.embeddings.word_embeddings.modules_to_save.default.weight

# from peft import PeftConfig, PeftModel
# config = PeftConfig.from_pretrained("models/XLNET/Model")
# model = PeftModel.from_pretrained(model, "models/XLNET/Model")

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params/1e6} || all params: {all_param/1e6} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
print_trainable_parameters(model)

trainable params: 635.376663 || all params: 635.376663 || trainable%: 100.00


In [ ]:
BATCH = 8
EPOCH = 2
ACCUMULATION = 8

training_args = TrainingArguments(
    output_dir="LLM",
    gradient_accumulation_steps = ACCUMULATION,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_steps = EPOCH * LEN // (BATCH * ACCUMULATION)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.122100,0.122422,0.861301,0.863487,0.862393,0.964304


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model = AutoModelForTokenClassification.from_pretrained("drive/MyDrive/NLP/Distilbert-original")
# tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/NLP/XLM-original")
# model.save_pretrained("drive/MyDrive/NLP/XLM-original")
# tokenizer.save_pretrained("drive/MyDrive/NLP/XLM-original")
print()
model

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(98562, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
model.distilbert.transformer.layer[0]

TransformerBlock(
  (attention): MultiHeadSelfAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (q_lin): Linear(in_features=768, out_features=768, bias=True)
    (k_lin): Linear(in_features=768, out_features=768, bias=True)
    (v_lin): Linear(in_features=768, out_features=768, bias=True)
    (out_lin): Linear(in_features=768, out_features=768, bias=True)
  )
  (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (ffn): FFN(
    (dropout): Dropout(p=0.1, inplace=False)
    (lin1): Linear(in_features=768, out_features=3072, bias=True)
    (lin2): Linear(in_features=3072, out_features=768, bias=True)
    (activation): GELUActivation()
  )
  (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
)

In [ ]:
import pandas as pd
original = pd.DataFrame()

In [ ]:
import gc
from tqdm import tqdm
def calculate_metrics(data):
    predictions=[]
    labels=[]
    for i in tqdm(range(0,len(data),700)):
        sample =  tokenize_and_align_labels(data[i:i+700])
        sample['input_ids']=sample['input_ids'].to("cuda")
        sample['attention_mask']=sample['attention_mask'].to("cuda")
        labels += sample['labels']
        # print(sample['input_ids'].shape)
        sample.pop('labels')

        with torch.no_grad():
            outputs = model(**sample).logits.tolist()
            outputs = np.argmax(outputs, axis=-1)
            predictions += outputs.tolist()
    gc.collect()
    xx = (predictions, labels)
    # print(np.array(predictions).shape)
    return compute_metrics(xx)

reply = calculate_metrics(hiner['test'])
reply

100%|██████████| 31/31 [03:07<00:00,  6.05s/it]


{'precision': 0.8500919160242844,
 'recall': 0.8561759992531857,
 'f1': 0.8531231105529976,
 'accuracy': 0.9627281603428168}

In [ ]:
reply['model']="DISTILBERT"
reply['split']="test"

reply = {i:[j] for i,j in reply.items()}

original = pd.concat([original, pd.DataFrame(reply)], ignore_index=True)
original

,precision,recall,f1,accuracy,model,split
0,0.926370,0.931331,0.928844,0.983636,DISTILBERT,train
1,0.849920,0.853181,0.851548,0.962128,DISTILBERT,validation
2,0.850092,0.856176,0.853123,0.962728,DISTILBERT,test


In [ ]:
original.to_excel("drive/MyDrive/NLP/originallll.xlsx", index=None)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
CHARLIST=[2305, 2306, 2307, 2364, 2366, 2367, 2368, 2369, 2370, 2371, 2372, 2373, 2374, 2375, 2376, 2377, 2378, 2379, 2380, 2381, 2402, 2403]
def custom_ner_pipeline(model_name, tokenizer, input_strings):
    inputs = tokenizer(input_strings, padding=True, truncation=True, return_tensors="pt", return_token_type_ids=False)
    print(inputs)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_labels = torch.argmax(outputs.logits, dim=2).tolist()
    ans = []
    for idx in range(len(predicted_labels)):
        word_ids = inputs.word_ids(batch_index = idx)
        entities = []
        current_entity = {"text": "", "label": ""}
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][idx])
        for i in range(len(inputs['input_ids'][idx])):
            token = tokens[i]
            label = model.config.id2label[predicted_labels[idx][i]]
            if word_ids[i]!=None:
                if label.startswith("B-"):
                    if current_entity["text"]:
                        entities.append(current_entity)
                    current_entity = {"text": token, "label": label[2:]}
                elif label.startswith("I-"):
                    if current_entity["text"]:
                        if token.startswith("##"):
                            current_entity["text"] += token[2:]
                        else:
                            current_entity["text"] += token
                    else:
                        current_entity = {"text": token, "label": label[2:]}
                elif label.startswith("O"):
                    if current_entity["text"]:
                        if token.startswith("##"):
                            current_entity["text"] += token[2:]
                        else:
                            current_entity["text"] += token
                    else:
                        current_entity = {"text": token, "label": "O"}
                if i < len(word_ids) - 1 and (word_ids[i] == word_ids[i + 1] or ord(tokens[i+1][0]) in CHARLIST):
                    pass
                else:
                    entities.append(current_entity)
                    current_entity = {"text": "", "label": ""}
        ans.append(entities)
    return ans

input_strings = [" ".join(hiner["train"]['tokens'][0])," ".join(hiner["train"]['tokens'][1]) ]

entities = custom_ner_pipeline(model, tokenizer, input_strings)
print(entities)


{'input_ids': tensor([[    3,  1915, 11845,  1899,  2171,  2250, 42205,   498,  1005,  1934,
          3402,  1942,  1956,  1900,  2271,  1956,  1952,  1915, 11845,  1899,
         25365,  5080,  2121, 63514,   445,  1886,  1952,  2139,  3733,  2255,
          1945,  2598,  1904, 39521,  1925, 83947,   485,  1886,    18,     4],
        [    3,  7101,  4331,  2053, 56882, 73480,  1889,  1940, 22676,  2548,
          1886,   507,     4,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


RuntimeError: ignored

In [ ]:
# demo =  model.merge_and_unload()

In [ ]:
# model_id = "./drive/MyDrive/NLP/DistilBERT/Model"
# model.save_pretrained(model_id)
# tokenizer.save_pretrained("./drive/MyDrive/NLP/DistilBERT/Tokenizer/")

# !zip -r drive/MyDrive/LLM/DistilBERT.zip ./DistilBERT

# from google.colab import files
# files.download("DistilBERT.zip")

In [ ]:
# !unzip drive/MyDrive/XLNET
# !unzip drive/MyDrive/DistilBERT
# !unzip drive/MyDrive/ConvBERT

In [ ]:
# model = AutoModelForTokenClassification.from_pretrained("XLNET/Model").cuda()
# tokenizer = AutoTokenizer.from_pretrained("XLNET/Tokenizer")

In [ ]:
# ['id', 'ner_tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels']
# tokenized = tokenizer(df['textwithoutbt'].loc[:100].tolist(), padding=True, return_tensors="pt")['input_ids'].cuda()
# model(tokenized)

In [ ]:
# from transformers import pipeline
# classifier = pipeline("ner", model=model, tokenizer = tokenizer, device="cuda")